In [45]:
import pandas as pd
import numpy as np

In [6]:
pri = pd.read_csv("../abelee_data/prices_primary.csv", delimiter=" ")
sec = pd.read_csv("../abelee_data/prices_secondary.csv", delimiter=" ")

In [15]:
sec.dtypes

timestamp                int64
datetime        datetime64[ns]
bid_price              float64
bid_amount             float64
offer_price            float64
offer_amount           float64
dtype: object

In [16]:
sec["datetime"] = pd.to_datetime(sec["datetime"])
pri["datetime"] = pd.to_datetime(pri["datetime"])

In [ ]:
sec.set_index("datetime", inplace=True)
pri.set_index("datetime", inplace=True)

In [25]:
sec.head()

,timestamp,bid_price,bid_amount,offer_price,offer_amount
datetime,,,,,
2021-07-30 00:00:00.000101406,1627603200000101406,1.18902,1500000.0,1.18905,1000000.0
2021-07-30 00:00:00.089534545,1627603200089534545,1.18902,1500000.0,1.18905,1500000.0
2021-07-30 00:00:00.090456940,1627603200090456940,1.18902,1000000.0,1.18905,1500000.0
2021-07-30 00:00:00.099915720,1627603200099915720,1.18901,1500000.0,1.18905,1500000.0
2021-07-30 00:00:00.099916479,1627603200099916479,1.18901,1500000.0,1.18904,1000000.0


In [31]:
mid_pri = (pri["bid_price"]+pri["offer_price"])/2
mid_sec = (sec["bid_price"]+sec["offer_price"])/2

In [35]:
mid_pri

datetime
2021-07-30 00:00:00.015116310    1.189025
2021-07-30 00:00:00.095110468    1.189025
2021-07-30 00:00:00.225114905    1.189050
2021-07-30 00:00:00.230111953    1.189050
2021-07-30 00:00:00.295118845    1.189025
                                   ...   
2021-07-30 20:59:40.914978098    1.186775
2021-07-30 20:59:40.924992001    1.186800
2021-07-30 20:59:40.934971541    1.186775
2021-07-30 20:59:56.084994115    1.187225
2021-07-30 20:59:56.094987391    1.187200
Length: 99207, dtype: float64

In [39]:
mid_pri_hours = mid_pri.resample(rule = pd.Timedelta(value = 1, unit="hours")).mean()
mid_sec_hours = mid_sec.resample(rule = pd.Timedelta(value = 1, unit="hours")).mean()

In [41]:
mid_pri_hours.corr(mid_sec_hours)

0.9996372837084017

In [42]:
mid_pri_hours

datetime
2021-07-30 00:00:00    1.188588
2021-07-30 01:00:00    1.188168
2021-07-30 02:00:00    1.188189
2021-07-30 03:00:00    1.188036
2021-07-30 04:00:00    1.187888
2021-07-30 05:00:00    1.187751
2021-07-30 06:00:00    1.188060
2021-07-30 07:00:00    1.189094
2021-07-30 08:00:00    1.189905
2021-07-30 09:00:00    1.190281
2021-07-30 10:00:00    1.189479
2021-07-30 11:00:00    1.189541
2021-07-30 12:00:00    1.188625
2021-07-30 13:00:00    1.188008
2021-07-30 14:00:00    1.186666
2021-07-30 15:00:00    1.186041
2021-07-30 16:00:00    1.186095
2021-07-30 17:00:00    1.185986
2021-07-30 18:00:00    1.185765
2021-07-30 19:00:00    1.185968
2021-07-30 20:00:00    1.186596
Freq: H, dtype: float64

In [54]:
mid_pri_hours_diff = np.log(mid_pri_hours).diff()[1:]
mid_sec_hours_diff = np.log(mid_sec_hours).diff()[1:]

In [55]:
mid_pri_hours_diff.head()

datetime
2021-07-30 01:00:00   -0.000354
2021-07-30 02:00:00    0.000018
2021-07-30 03:00:00   -0.000129
2021-07-30 04:00:00   -0.000124
2021-07-30 05:00:00   -0.000116
Freq: H, dtype: float64

In [56]:
mid_pri_hours_diff.corr(mid_sec_hours_diff)

0.9953375473912728

In [65]:
def corr(t_1, t_2, t_delta_val, t_delta_unit):
    t_1_re = t_1.resample(rule = pd.Timedelta(value = t_delta_val, unit=t_delta_unit)).last()
    t_2_re = t_2.resample(rule = pd.Timedelta(value = t_delta_val, unit=t_delta_unit)).last()
    t_1_re = np.log(t_1_re).diff()[1:]
    t_2_re = np.log(t_2_re).diff()[1:]
    return(t_1_re.corr(t_2_re))

In [67]:
corr(t_1 = mid_pri, t_2 = mid_sec, t_delta_val = 1, t_delta_unit="hours")

0.9895063849615194